<a href="https://colab.research.google.com/github/ClaudioLucioLopes/PrescriptiveAnalysis/blob/master/3_0_Projeto_de_757_Mobility_Startup_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso da Projeto 757 Mobility Startup

# Instalação e módulos necessários

In [0]:
!pip install ortools
from ortools.linear_solver import pywraplp
import numpy as np


     |████████████████████████████████| 27.7MB 84kB/s 


# Organizando os dados

In [0]:


#Tempo calculado a partir da ultima atualização de tráfego e posição dos motorstas
Tempo_Previsto = np.array([[90, 76, 75, 70],
                           [35, 85, 55, 65],
                           [125, 95, 90, 105],
                           [45, 110, 95, 115],
                           [60, 105, 80, 75],
                           [45, 65, 110, 95]])

"""Tempo_Previsto = np.array([[90, 76, 75, 70],
                           [35, 85, 55, 65],
                           [125, 95, 90, 105],
                           [45, 110, 95, 115],
                           [60, 105, 80, 75],
                           [45, 65, 110, 95]])
"""

#Número de clientes que solicitaram o serviço no bairro naquele momento
numero_clientes = Tempo_Previsto.shape[0]
numero_motoristas = Tempo_Previsto.shape[1]



# Criando o modelo

In [0]:
# Criando o modelo de atendimento motoristas e clientes
modelo_atendimento = pywraplp.Solver('Modelo de atendimento',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


#Criando as variáveis de decisão
atendimento = {}

for i in range(numero_clientes):
    for j in range(numero_motoristas):
        atendimento[i, j] = modelo_atendimento.BoolVar('atendimento[%i,%i]' % (i, j))

#Criando a nossa função objetivo e atribuindo a minimizaçãão
modelo_atendimento.Minimize(modelo_atendimento.Sum(Tempo_Previsto[i,j] * atendimento[i,j] for i in range(numero_clientes) for j in range(numero_motoristas)))


#Restrições para os atendimentos dos clientes e dos motoristas


# Cada cliente deve ser atendido por apenas 1 motorista .
if numero_motoristas < numero_clientes:
    for i in range(numero_clientes):
        modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for j in range(numero_motoristas)]) <= 1)
else:
    for i in range(numero_clientes):
        modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for j in range(numero_motoristas)]) == 1)
    

# Se o número de clientes for maior que o número de motoristas vamos garantir que os atendimentos mais rápidos aconteçam.
if numero_motoristas < numero_clientes:
    modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for i in range(numero_clientes) for j in range(numero_motoristas)]) >= numero_motoristas)

# Cada motorista deve atender no máximo 1 pessoa naquele instante de tempo .
for j in range(numero_motoristas):
    modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for i in range(numero_clientes)]) <= 1)





# Executando o modelo

In [0]:
#Executando o modelo
modelo_atendimento.Solve()

print('Tempo: ', modelo_atendimento.WallTime(), 'ms')
print('Iterações: ', modelo_atendimento.iterations())

Tempo:  19 ms
Iterações:  0


# Avaliando os resultados

In [0]:
print('Tempo total para os atendimentos: ', round(modelo_atendimento.Objective().Value(),2))

print()
for i in range(numero_clientes):
    for j in range(numero_motoristas):
        #print(atendimento[i, j].solution_value() )
        if atendimento[i, j].solution_value() > 0:
            print('Cliente %d será atendido pelo motorista %d.  Tempo previsto = %d' % (i+1,j+1,Tempo_Previsto[i,j]))
print()



Tempo total para os atendimentos:  235.0

Cliente 1 será atendido pelo motorista 4.  Tempo previsto = 70
Cliente 2 será atendido pelo motorista 3.  Tempo previsto = 55
Cliente 4 será atendido pelo motorista 1.  Tempo previsto = 45
Cliente 6 será atendido pelo motorista 2.  Tempo previsto = 65

